In [91]:
#Importing required libraries

from bs4 import BeautifulSoup
import requests
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
#import chromedriver_binary
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.select import Select

import requests
import json
import pymongo


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

header = { 'User-Agent':
           'Mozilla/5.0 (X11; windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
                           'Accept-Language': 'en-US, en;q=0.5'}
entriesinfo=[] #To add stock abbreviations
stocknames = [] #To add stock names

driver.get("https://www.valueinvestorsclub.com/login");

#Logging into the portal to access 45 day data instead of 90 day old data

username =  driver.find_element_by_css_selector("input[type=text]")
password = driver.find_element_by_css_selector("input[type=Password]")

username.send_keys("username")
time.sleep(3)
password.send_keys("password here")
time.sleep(3)

driver.find_element_by_id("login_btn").click()

time.sleep(5)

#Opening the website - now logged in

driver.get("https://www.valueinvestorsclub.com/ideas#i9");
driver.refresh()

input_el = driver.find_element_by_id('ideas_loc_text')
td_p_input = input_el.find_element_by_xpath('..')

WebDriverWait(driver,20).until(expected_conditions.element_to_be_clickable((td_p_input))).click()

# Filtering for US only as we are focussing only on US stocks

driver.find_element(By.XPATH, "//a[text()='US Only']").click();

time.sleep(10)

entries = driver.find_elements_by_class_name("entry-header")


#Storing each stock name and symbol in a list

for value in entries:
        ent=value.text
        s1=ent.split('•')[0]
        
        if s1.split(' ')[-2] == 'US':
            s2=s1.split(' ')[-3]
        else:
            s2=s1.split(' ')[-2]
        
        entriesinfo.append(s2) #Abbreviations
        stocknames.append(s1) #Stock names
        
driver.close() #Closing the Selenium driver

#Removing duplicates as some stocks tend to re-appear in the homepage

entriesinfo = list(dict.fromkeys(entriesinfo))
stocknames = list(dict.fromkeys(stocknames))







====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/yashasbvn/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
<ipython-input-91-3e5e2d07a30a>:32: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username =  driver.find_element_by_css_selector("input[type=text]")
<ipython-input-91-3e5e2d07a30a>:33: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_css_selector("input[type=Password]")
<ipython-input-91-3e5e2d07a30a>:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("login_btn").click()
<ipython-input-91-3e5e2d07a30a>:49: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  input_el = driver.find_element_by_id('

In [92]:
#Printing the stock symbols
print(entriesinfo)

#Printing the stock names
print(stocknames)

['LCUT', 'SSTK', 'FCNE', 'Z', 'TV', 'TLYS', 'SAH', 'EPC', 'VQS', 'GNE', 'MSI', 'LAWS', 'AMLX', 'EXC', 'ALIT', 'NWLI', 'GIL.', 'media', 'LEN', 'AJRD', 'whlrl', 'SCWX', 'ARNA', 'WED.', 'ACVA', 'TIGO', 'CRVS', 'AMZN', 'COMP', 'VZ', 'DLO', 'SONO', 'BBWI', 'ROKU', 'WMT', 'MHH', 'RDI', 'ECNCF', 'ACT', 'AMNF', 'SNP', 'WLMS', 'COST', 'MNKKQ', 'doug', 'PGR', 'VSH', 'WEX', 'AFHBL', 'WE', 'ELVT', 'FLR', 'RKLY', 'VEEE', 'AGO', 'SBRA', 'ASO', 'ZENV', 'MHK', 'HAIN', 'MIXT', 'FTAI', 'AZPN', 'GDOT', 'EXPR', 'GRPN', 'BMTX', 'LYLT', 'DIBS', 'AZTA', 'BDC', 'FTDR', 'LUB', 'PYPL', 'CVS', 'PLXP', 'LTSF', 'TACT', 'OPT', 'RIVN', 'FHFA', 'BV', 'BVS', 'KOD', 'TSAT', 'GLASF', 'ACTDU', 'IIIV', 'GOED', 'GTBIF']
['LIFETIME BRANDS INC LCUT ', 'SHUTTERSTOCK INC SSTK ', 'FOUR CORNERS INC FCNE ', 'ZILLOW GROUP INC Z ', 'GRUPO TELEVISA SAB TV ', "TILLY'S INC TLYS ", 'Sonic Automotive SAH ', 'EDGEWELL PERSONAL CARE CO EPC ', 'VIQ Solutions VQS ', 'GENIE ENERGY LTD GNE ', 'MOTOROLA SOLUTIONS INC MSI ', 'LAWSON PRODUCTS LA

In [93]:
#Accessing Rapid Yahoo Finance API to get the stock details of each of the above stocks

stockdetails =[]

url = "https://yh-finance.p.rapidapi.com/stock/v2/get-summary"

#Getting info of each stock

for i in range(0,len(entriesinfo)):

    querystring = {"symbol":entriesinfo[i],"region":"US"} #The parameters that we pass into the API

    headers = {
        'x-rapidapi-host': "yh-finance.p.rapidapi.com",
        'x-rapidapi-key': "4ac4991187msh6e9c5a8cb610d9dp10e9a9jsn3c5fad87e91a"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    #Adding each stock data into a dictionary
    
    try:
        loc1 = json.loads(response.text)["financialData"]
        stockdetails.append(loc1)
    except:
        stockdetails.append("") #Appending empty string if no data available

#Printing the stock details
print(stockdetails)

[{'ebitdaMargins': {'raw': 0.10342, 'fmt': '10.34%'}, 'profitMargins': {'raw': 0.0241, 'fmt': '2.41%'}, 'grossMargins': {'raw': 0.3515, 'fmt': '35.15%'}, 'operatingCashflow': {'raw': 36988000, 'fmt': '36.99M', 'longFmt': '36,988,000'}, 'revenueGrowth': {'raw': 0.027, 'fmt': '2.70%'}, 'operatingMargins': {'raw': 0.07732, 'fmt': '7.73%'}, 'ebitda': {'raw': 89245000, 'fmt': '89.25M', 'longFmt': '89,245,000'}, 'targetLowPrice': {'raw': 26.5, 'fmt': '26.50'}, 'recommendationKey': 'buy', 'grossProfits': {'raw': 303319000, 'fmt': '303.32M', 'longFmt': '303,319,000'}, 'freeCashflow': {'raw': 31594376, 'fmt': '31.59M', 'longFmt': '31,594,376'}, 'targetMedianPrice': {'raw': 26.75, 'fmt': '26.75'}, 'currentPrice': {'raw': 13.1, 'fmt': '13.10'}, 'earningsGrowth': {}, 'currentRatio': {'raw': 2.264, 'fmt': '2.26'}, 'returnOnAssets': {'raw': 0.05096, 'fmt': '5.10%'}, 'numberOfAnalystOpinions': {'raw': 2, 'fmt': '2', 'longFmt': '2'}, 'targetMeanPrice': {'raw': 26.75, 'fmt': '26.75'}, 'debtToEquity': {

In [94]:
#Appending stock names and symbols to each of the stocks above

stock_dict = []

for i in range(0,len(entriesinfo)):
    stockname_dict = {'stock':entriesinfo[i], 'stock_name': stocknames[i]}
    
    #Appending the stock names to other details
    stockname_dict.update(stockdetails[i])
    
    stock_dict.append(stockname_dict) 

print(stock_dict)

<class 'dict'>
[{'stock': 'LCUT', 'stock_name': 'LIFETIME BRANDS INC LCUT ', 'ebitdaMargins': {'raw': 0.10342, 'fmt': '10.34%'}, 'profitMargins': {'raw': 0.0241, 'fmt': '2.41%'}, 'grossMargins': {'raw': 0.3515, 'fmt': '35.15%'}, 'operatingCashflow': {'raw': 36988000, 'fmt': '36.99M', 'longFmt': '36,988,000'}, 'revenueGrowth': {'raw': 0.027, 'fmt': '2.70%'}, 'operatingMargins': {'raw': 0.07732, 'fmt': '7.73%'}, 'ebitda': {'raw': 89245000, 'fmt': '89.25M', 'longFmt': '89,245,000'}, 'targetLowPrice': {'raw': 26.5, 'fmt': '26.50'}, 'recommendationKey': 'buy', 'grossProfits': {'raw': 303319000, 'fmt': '303.32M', 'longFmt': '303,319,000'}, 'freeCashflow': {'raw': 31594376, 'fmt': '31.59M', 'longFmt': '31,594,376'}, 'targetMedianPrice': {'raw': 26.75, 'fmt': '26.75'}, 'currentPrice': {'raw': 13.1, 'fmt': '13.10'}, 'earningsGrowth': {}, 'currentRatio': {'raw': 2.264, 'fmt': '2.26'}, 'returnOnAssets': {'raw': 0.05096, 'fmt': '5.10%'}, 'numberOfAnalystOpinions': {'raw': 2, 'fmt': '2', 'longFmt':

In [95]:
#Pretty printing the stock details

print(json.dumps(stock_dict, indent=4, sort_keys=False))

[
    {
        "stock": "LCUT",
        "stock_name": "LIFETIME BRANDS INC LCUT ",
        "ebitdaMargins": {
            "raw": 0.10342,
            "fmt": "10.34%"
        },
        "profitMargins": {
            "raw": 0.0241,
            "fmt": "2.41%"
        },
        "grossMargins": {
            "raw": 0.3515,
            "fmt": "35.15%"
        },
        "operatingCashflow": {
            "raw": 36988000,
            "fmt": "36.99M",
            "longFmt": "36,988,000"
        },
        "revenueGrowth": {
            "raw": 0.027,
            "fmt": "2.70%"
        },
        "operatingMargins": {
            "raw": 0.07732,
            "fmt": "7.73%"
        },
        "ebitda": {
            "raw": 89245000,
            "fmt": "89.25M",
            "longFmt": "89,245,000"
        },
        "targetLowPrice": {
            "raw": 26.5,
            "fmt": "26.50"
        },
        "recommendationKey": "buy",
        "grossProfits": {
            "raw": 303319000,
       

In [96]:
############ Inserting into MongoDB server ############

client = pymongo.MongoClient("mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false")
mydb = client["value_investors"] #Creating a database if it does not exist already
mycol = mydb["stock_details"] #Choosing the collection where data would be input

#Inserting into the collection on MongoDB
x = mycol.insert_many(stock_dict)

print("Results exported to MongoDB.")


Results exported to MongoDB.
